In [1]:
import os
from dotenv import load_dotenv
import requests

# .env 파일 로드
load_dotenv()

# API 키 가져오기
api_key = os.getenv("API_KEY")


In [ ]:
import requests
 
filename = r"C:\Users\ld115\OneDrive\바탕 화면\업스\cookbook\Solar-Fullstack-LLM-101\pdfs\예시 ir.pdf" #tips PDF 파일 경로
 
url = "https://api.upstage.ai/v1/document-ai/ocr"
headers = {"Authorization": f"Bearer {api_key}"}
files = {"document": open(filename, "rb")}
response = requests.post(url, headers=headers, files=files)
print(response.json())

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\ld115\\OneDrive\\바탕 화면\\업스\\cookbook\\Solar-Fullstack-LLM-101\\pdfs\\예시 ir.pdf'

In [ ]:
import requests
import PyPDF2
from io import BytesIO

# API 정보
ocr_url = "https://api.upstage.ai/v1/document-ai/ocr"
solar_url = "https://api.upstage.ai/v1/solar/chat/completions"
headers = {"Authorization": f"Bearer {api_key}"}

def split_pdf(filename, pages_per_chunk=5):
    """PDF 파일을 pages_per_chunk만큼 나눠서 BytesIO 객체 리스트로 반환."""
    pdf_chunks = []
    with open(filename, "rb") as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        total_pages = len(pdf_reader.pages)

        for start in range(0, total_pages, pages_per_chunk):
            pdf_writer = PyPDF2.PdfWriter()
            for i in range(start, min(start + pages_per_chunk, total_pages)):
                pdf_writer.add_page(pdf_reader.pages[i])

            pdf_chunk = BytesIO()
            pdf_writer.write(pdf_chunk)
            pdf_chunk.seek(0)
            pdf_chunks.append(pdf_chunk)

    return pdf_chunks

def parse_pdf_and_generate_individual_plans(filename):
    pdf_chunks = split_pdf(filename, pages_per_chunk=5)

    for chunk_num, pdf_chunk in enumerate(pdf_chunks):
        files = {"document": pdf_chunk}
        response = requests.post(ocr_url, headers=headers, files=files)

        if response.status_code != 200:
            print(f"OCR API 호출 실패: {response.status_code}")
            continue
        
        response_data = response.json()
        
        for page_num, page in enumerate(response_data.get("pages", [])):
            page_text = page.get("text", "")

            prompt = f"""You are a CEO drafting a business plan. Based on the following text, please write a comprehensive business plan in Korean according to the given format in Markdown.

### 요구사항:
{page_text}

### 양식:
1-1. **창업아이템의 고도화 동기**
- 외적 동기(사회·경제·기술 분야 국내·외 시장의 문제점·기회 등)
- 내적 동기(대표자 및 자사의 보유역량, 가치관, 비전 등)의 관점에서 제품·서비스를 고도화하게 된 동기를 기재
- 현재 보유한 제품·서비스의 매출 실현 한계점 및 신제품 개발 필요성 등을 반영하여 기재

1-2. **창업아이템의 고도화 목적**
- 현재 시점(본 지원사업 신청 시점)의 제품·서비스 개발 상태 등 설명
- 고도화 동기에서 발견한 문제점·기회에 대해 해당 제품·서비스가 제시할 수 있는 혁신적인 해결방안, 고도화 필요성, 고객들에게 제공할 혜택 등의 관점에서 제품·서비스를 고도화하려는 목적을 기재

1-3. **창업아이템의 목표시장 분석**
- 국내·외 진입시장과 확장시장의 규모·상황 및 특성, 현재 위치, 경쟁 강도, 향후 전망(성장성), 주요 고객 특성 등 기재

1-4. **창업아이템의 고도화를 위한 개선 과제**
  - 1-4-1. 제품·서비스에 대한 개선 과제
  - 1-4-2. 고객 요구사항에 대한 개선 과제
  - 1-4-3. 경쟁제품·서비스 대비 개선 과제

2. **실현가능성(Solution)**
  - 2-1. 창업아이템의 고도화 방안
  - 2-2. 고객 요구사항 대응 방안
  - 2-3. 창업아이템의 차별화 강화 방안

3. **성장전략(Scale-up)**
  - 3-1. 창업아이템의 사업화 방안
    - 3-1-1. 비즈니스 모델(BM)
    - 3-1-2. 목표시장 진입 현황

### 마지막으로 스스로 확인:
0. 분량은 적절히 많은가?
1. 간단하게 소주제로 설득력있는가?
2. 에세이같은 말투가 아니라 정형화된 말투인가?
3. 근거를 들어서 설명했는가?
4. 점수 제시?
5. 출처가 명확한가?
6. 요약체로 정리하였는가?

### 추가 질문들:
1. 외적동기가 명확한가? 
2. 내적동기가 이상 없이 설명되었는가?
3. 제품의 공신력을 증명할 부분이 있는가?
4. 소비자 니즈를 만족시키는 결과가 있는가?
5. 기간 내 완성도가 보장되는가?
6. 시장의 전체 규모와 목표 규모는 적절한가?
"""

            solar_payload = {
                "model": "solar-mini",
                "messages": [
                    {
                        "role": "user",
                        "content": prompt
                    }
                ],
                "stream": False
            }
            
            solar_response = requests.post(solar_url, headers=headers, json=solar_payload)
            if solar_response.status_code != 200:
                print(f"SOLAR API 호출 실패: {solar_response.status_code}")
                continue
            
            solar_data = solar_response.json()
            
            output_file = f"business_plan_chunk_{chunk_num + 1}_page_{page_num + 1}.md"
            with open(output_file, "w", encoding="utf-8") as file:
                if "choices" in solar_data:
                    file.write(solar_data["choices"][0]["message"]["content"])
                print(f"청크 {chunk_num + 1}, 페이지 {page_num + 1}의 사업계획서가 '{output_file}' 파일에 저장되었습니다.")

        pdf_chunk.close()

filename = r"C:\Users\ld115\OneDrive\바탕 화면\업스\cookbook\Solar-Fullstack-LLM-101\pdfs\국문IR.pdf"  # tips PDF 파일 경로
parse_pdf_and_generate_individual_plans(filename)


청크 1, 페이지 1의 사업계획서가 'business_plan_chunk_1_page_1.md' 파일에 저장되었습니다.
청크 1, 페이지 2의 사업계획서가 'business_plan_chunk_1_page_2.md' 파일에 저장되었습니다.
청크 1, 페이지 3의 사업계획서가 'business_plan_chunk_1_page_3.md' 파일에 저장되었습니다.
청크 1, 페이지 4의 사업계획서가 'business_plan_chunk_1_page_4.md' 파일에 저장되었습니다.
청크 1, 페이지 5의 사업계획서가 'business_plan_chunk_1_page_5.md' 파일에 저장되었습니다.
청크 2, 페이지 1의 사업계획서가 'business_plan_chunk_2_page_1.md' 파일에 저장되었습니다.
청크 2, 페이지 2의 사업계획서가 'business_plan_chunk_2_page_2.md' 파일에 저장되었습니다.
청크 2, 페이지 3의 사업계획서가 'business_plan_chunk_2_page_3.md' 파일에 저장되었습니다.
청크 2, 페이지 4의 사업계획서가 'business_plan_chunk_2_page_4.md' 파일에 저장되었습니다.
청크 2, 페이지 5의 사업계획서가 'business_plan_chunk_2_page_5.md' 파일에 저장되었습니다.
청크 3, 페이지 1의 사업계획서가 'business_plan_chunk_3_page_1.md' 파일에 저장되었습니다.
청크 3, 페이지 2의 사업계획서가 'business_plan_chunk_3_page_2.md' 파일에 저장되었습니다.
청크 3, 페이지 3의 사업계획서가 'business_plan_chunk_3_page_3.md' 파일에 저장되었습니다.
청크 3, 페이지 4의 사업계획서가 'business_plan_chunk_3_page_4.md' 파일에 저장되었습니다.
청크 3, 페이지 5의 사업계획서가 'business_plan_chunk_3_page_5.md' 파일에 저장되었

In [ ]:
from transformers import AutoTokenizer
from langchain.text_splitter import TokenTextSplitter

solar_tokenizer = AutoTokenizer.from_pretrained("upstage/solar-pro-preview-instruct")

from langchain.schema import Document

docs = []
for i in range(1, 4):  
    for j in range(1, 6):  
        filename = f"business_plan_chunk_{i}_page_{j}.md"
        try:
            with open(filename, 'r', encoding='utf-8') as f:
                content = f.read()
            doc = Document(page_content=content)
            docs.append(doc)
        except FileNotFoundError:
            print(f"파일 {filename}을(를) 찾을 수 없습니다.")

token_splitter = TokenTextSplitter.from_huggingface_tokenizer(
    solar_tokenizer, chunk_size=250, chunk_overlap=100
)

splits = token_splitter.split_documents(docs)
print(f"분할된 문서 수: {len(splits)}")

from langchain_upstage import UpstageEmbeddings
from langchain.vectorstores import Chroma

embeddings = UpstageEmbeddings(model="solar-embedding-1-large")

db = Chroma(embedding_function=embeddings)

db.add_documents(splits)

retriever = db.as_retriever()

from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_upstage import ChatUpstage

llm = ChatUpstage()

user_business_plan = """
1-1. 창업아이템의 고도화 동기
외적 동기: 현재 국내외 스타트업 생태계는 다양한 프로그램과 자금 지원을 통해 창업을 장려하고 있으나, 많은 창업자들은 사업계획서 작성 과정에서 어려움을 겪고 있다. 특히, TIPS와 같은 고도화된 지원 프로그램에 맞춰서 사업계획서를 작성하는 것은 처음 창업자들에게 높은 진입장벽으로 작용한다. 또한 기존의 선정된 스타트업 사례를 통해 분석적 접근을 하여 성공 가능성을 높이는 프로그램의 필요성이 점점 증가하고 있다.

내적 동기: 본 프로그램 개발자는 다양한 IR 자료를 분석할 수 있는 역량을 보유하고 있으며, 이를 바탕으로 AI와 자연어 처리를 통해 효과적인 사업계획서 피드백 시스템을 구축할 수 있는 기술력을 갖추고 있다. 창업자들이 더 경쟁력 있는 사업계획서를 작성할 수 있도록 도움을 주고자 하는 비전이 본 제품·서비스의 고도화 동기로 작용한다.

1-2. 창업아이템의 고도화 목적
현재 본 프로그램은 기존 TIPS 선정 기업들의 IR 자료와 Solar가 생성한 사업계획서를 학습한 상태이다. 그러나 더욱 효과적인 피드백을 제공하기 위해 사용자 맞춤형 분석 및 추가 학습 기능이 필요하다. 본 프로그램은 사업계획서 작성자의 문서와 TIPS 기준의 최적화된 작성 방식 간의 차이를 자동으로 분석해 사용자에게 개선사항을 제시할 수 있다. 이를 통해 사용자는 사업계획서의 완성도를 높일 수 있으며, 사업 성공 가능성을 제고할 수 있다.

1-3. 창업아이템의 목표시장 분석
본 프로그램의 목표시장은 국내외 스타트업 및 창업자들로, 특히 TIPS와 같은 고도화된 지원 프로그램에 신청하는 이들이다. 스타트업 지원이 활발히 이루어지고 있는 국내외 시장에서 본 프로그램은 기존 사업계획서 작성 도구와 차별화된 분석 및 피드백 기능을 제공한다. 주요 고객은 창업 초기 단계에 있는 창업자 및 중소기업이며, 이들이 본 프로그램을 통해 더 나은 사업계획서로 경쟁력을 갖출 수 있도록 돕는 것을 목표로 한다.

1-4. 창업아이템의 고도화를 위한 개선 과제
1-4-1. 제품·서비스에 대한 개선 과제
현재 프로그램의 분석 및 피드백 기능은 특정 키워드 기반으로 작동하고 있으나, 사용자 맞춤형 피드백과 문서의 전체적인 구조 개선을 위해 더욱 정밀한 자연어 처리 기능이 필요하다.

1-4-2. 고객 요구사항에 대한 개선 과제
사용자 피드백을 통해, 다양한 사업 유형에 맞춘 더 구체적인 피드백이 필요함을 확인하였다. 창업자들의 다양한 산업군과 단계에 맞춘 피드백을 제공할 수 있도록 분석 기능을 고도화해야 한다.

1-4-3. 경쟁제품·서비스 대비 개선 과제
일반적인 사업계획서 작성 도구와 달리, 본 프로그램은 실제 TIPS 선정 사례에 기반한 맞춤형 분석을 제공하므로 이를 더욱 차별화하기 위한 보완이 필요하다.

2. 실현가능성 (Solution)
2-1. 창업아이템의 고도화 방안
본 프로그램은 협약 기간 내 Solar 기반의 IR 데이터를 추가 학습시키고, 사용자 맞춤형 피드백 기능을 구현하는 것을 목표로 한다. AI와 데이터 분석 인력을 확보하여 지속적인 기술 고도화를 진행할 예정이다. 또한, 사업계획서 작성 시 민감한 정보 보호를 위해 접근 권한 관리 및 보안 시스템을 운영할 예정이다.

2-2. 고객 요구사항 대응 방안
고객의 요구에 맞춘 맞춤형 피드백 제공을 위해 TIPS 선정 기업과 비선정 기업의 사업계획서를 비교하여 차별점과 공통점 분석을 통한 피드백 기능을 추가할 계획이다.

2-3. 창업아이템의 차별화 강화 방안
TIPS 선정 사례 기반의 데이터 분석을 통해 다른 경쟁 서비스와 차별화된 맞춤형 피드백을 제공하고, 이를 통해 사용자가 작성한 사업계획서의 완성도를 높일 수 있도록 돕는 것을 목표로 한다.

3. 성장전략 (Scale-up)
3-1. 창업아이템의 사업화 방안
3-1-1. 비즈니스 모델(BM)
본 프로그램은 월 구독 서비스와, 작성된 사업계획서의 검토를 제공하는 프리미엄 분석 서비스로 수익화를 목표로 한다.

3-1-2. 목표시장 진입 현황
현재 본 프로그램은 국내외 TIPS 지원자를 대상으로 시범 출시를 완료한 상태이다. 사용자의 긍정적인 피드백을 바탕으로 기능을 지속적으로 개선해나갈 계획이며, 향후 글로벌 시장으로의 진출도 검토 중이다.
"""

context = f"""
다음 사업계획서의 부족한 점을 찾아주십시오:
{user_business_plan}
기반으로 할 문서 내용:
{''.join([doc.page_content for doc in splits])}
"""

prompt_template = PromptTemplate.from_template(
    """
    {context}
    부족한 점을 명확히 지적하고, 개선 사항을 제안해주세요. 아주 자세하게 부탁합니다.
    다음 기준으로 평가 부탁합니다.
    
1. 간단하게 소주제로 설득력있는가? 
2. 에세이같은 말투가 아니라 정형화된 말투인가?
3. 근거를 들어서 설명했는가
4. 점수 제시?
5. 출처가 명확한가?
6. 요약체로 정리하였는가?


1. 사회, 경제, 기술 등 사람들이 문제해결을 위해 큰 비용을 들이는 것을 해결할 외적동기가 있는가? 
2. 외적동기에서 시장의 규모를 언급하면 제외
3. 글꼴 , 맑은 고딕,굴림체 추천/ 각주로 출처제시
4. 내적동기가 이상한 부분 없이 잘 설명 되었는가?
5. 제품 판매 공신력을 증명할만한 부분이 있는가?(특허, 저작권, 공구, 펀딩)
6. 소비자 니즈를 만족시킨 결과가 있는가?(sns)
7. 기간내의 완성도가 어느정도 보장되는가?
8. 시장의 규모는 전체 시장규모
9. 목표 규모는 구체적인 분야의 규모-정확한 통계와 출처인가?
    """
)

chain = prompt_template | llm | StrOutputParser()

advice = chain.invoke({"context": context})

print(advice)


분할된 문서 수: 319
제공된 사업계획서는 대부분의 섹션에서 간결하고 설득력 있는 내용을 담고 있습니다. 글쓰기 스타일은 에세이보다는 정형화된 비즈니스 문서 스타일에 가깝습니다. 근거를 제시하고, 요약체로 정리한 부분이 많습니다. 그러나 몇 가지 개선할 점이 있습니다.

1. 외적 동기에 대한 충분한 설명이 부족합니다. 사회, 경제, 기술 등 사람들이 문제해결을 위해 큰 비용을 들이는 것을 해결할 동기를 더욱 명확히 설명할 필요가 있습니다.
2. 내적 동기는 이상 없이 잘 설명되어 있습니다.
3. 제품 판매 공신력을 증명할만한 부분이 부족합니다. 특허, 저작권, 공구, 펀딩 등의 정보가 포함되어 있지 않습니다.
4. 소비자 니즈를 만족시킨 결과를 더욱 구체적으로 제시할 필요가 있습니다. SNS나 다른 플랫폼에서의 반응을 언급하는 것이 도움이 될 수 있습니다.
5. 기간 내의 완성도를 보장하기 위한 구체적인 계획이 부족합니다. 개발 일정과 위험 요소에 대한 대응책을 더욱 상세히 기술해야 합니다.
6. 시장의 전체 규모와 목표 규모에 대한 정확한 통계와 출처가 필요합니다.

전반적으로, 제공된 사업계획서는 좋은 내용을 담고 있지만, 몇 가지 개선 사항을 보완하여 더욱 강력하고 설득력 있는 문서로 만들 수 있을 것입니다.
